In [1]:
# import the necessary libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
# import datetime as dt

In [16]:
df = pd.read_excel('../datasets/cleaned_orders.xlsx')
print(df.dtypes)
df.head(2)

Unnamed: 0                                int64
customer_id                               int64
order_time                               object
item_quantity                             int64
item_id                                  object
item_price                              float64
item_discount_raw                       float64
item_discount_percentage                float64
item_disc_bins_item_disc_null             int64
item_disc_bins_item_disc_sub_twenty       int64
item_disc_bins_item_disc_sub_forty        int64
item_disc_bins_item_disc_sub_sixty        int64
item_disc_bins_item_disc_sub_eighty       int64
item_disc_bins_item_disc_sub_hundred      int64
dtype: object


,Unnamed: 0,customer_id,order_time,item_quantity,item_id,item_price,item_discount_raw,item_discount_percentage,item_disc_bins_item_disc_null,item_disc_bins_item_disc_sub_twenty,item_disc_bins_item_disc_sub_forty,item_disc_bins_item_disc_sub_sixty,item_disc_bins_item_disc_sub_eighty,item_disc_bins_item_disc_sub_hundred
0,0,0,2022-04-29 18:29:49 +0800,1,Arzuaga blanco,49.0,0.0,0.0,1,0,0,0,0,0
1,1,0,2022-04-29 18:29:49 +0800,1,Arzuaga reserva,94.0,0.0,0.0,1,0,0,0,0,0


In [17]:
# remove time from datetime so we can groupby later

df.drop(columns=df.columns[0], axis=1, inplace=True)
df['order_time'] = pd.to_datetime(df['order_time']).dt.date
df.head(2)

,customer_id,order_time,item_quantity,item_id,item_price,item_discount_raw,item_discount_percentage,item_disc_bins_item_disc_null,item_disc_bins_item_disc_sub_twenty,item_disc_bins_item_disc_sub_forty,item_disc_bins_item_disc_sub_sixty,item_disc_bins_item_disc_sub_eighty,item_disc_bins_item_disc_sub_hundred
0,0,2022-04-29,1,Arzuaga blanco,49.0,0.0,0.0,1,0,0,0,0,0
1,0,2022-04-29,1,Arzuaga reserva,94.0,0.0,0.0,1,0,0,0,0,0


In [18]:
df.columns

Index(['customer_id', 'order_time', 'item_quantity', 'item_id', 'item_price',
       'item_discount_raw', 'item_discount_percentage',
       'item_disc_bins_item_disc_null', 'item_disc_bins_item_disc_sub_twenty',
       'item_disc_bins_item_disc_sub_forty',
       'item_disc_bins_item_disc_sub_sixty',
       'item_disc_bins_item_disc_sub_eighty',
       'item_disc_bins_item_disc_sub_hundred'],
      dtype='object')

In [47]:
grouped = df[['order_time', 'item_id', 'item_quantity', 'item_price', 
              'item_disc_bins_item_disc_null', 
              'item_disc_bins_item_disc_sub_twenty',
              'item_disc_bins_item_disc_sub_forty',
              'item_disc_bins_item_disc_sub_sixty',
              'item_disc_bins_item_disc_sub_eighty',
              'item_disc_bins_item_disc_sub_hundred']]

In [48]:
groupedbyitem = grouped.groupby('item_id', group_keys=True).apply(lambda x:x)


In [53]:
groupedbyitem['item_id'].value_counts()
sample = groupedbyitem[groupedbyitem['item_id']=='Rara Neagră de Purcari 2020']

### Multivariate timeseries analysis using VAR (vector auto regression) - for one wine

In [54]:
sample.head(2)

,order_time,item_id,item_quantity,item_price,item_disc_bins_item_disc_null,item_disc_bins_item_disc_sub_twenty,item_disc_bins_item_disc_sub_forty,item_disc_bins_item_disc_sub_sixty,item_disc_bins_item_disc_sub_eighty,item_disc_bins_item_disc_sub_hundred
288,2022-07-24,Rara Neagră de Purcari 2020,1,50.0,1,0,0,0,0,0
301,2022-07-07,Rara Neagră de Purcari 2020,1,50.0,1,0,0,0,0,0


In [55]:
sample.drop(columns=sample.columns[1], axis=1, inplace=True)
sample = sample.groupby('order_time', group_keys=True).apply(lambda x:x)

/var/folders/j9/hl0rz3wj6yb5zwjtn6qjs8bh0000gn/T/ipykernel_1279/2557197964.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample.drop(columns=sample.columns[1], axis=1, inplace=True)


In [57]:
sample.columns

Index(['order_time', 'item_quantity', 'item_price',
       'item_disc_bins_item_disc_null', 'item_disc_bins_item_disc_sub_twenty',
       'item_disc_bins_item_disc_sub_forty',
       'item_disc_bins_item_disc_sub_sixty',
       'item_disc_bins_item_disc_sub_eighty',
       'item_disc_bins_item_disc_sub_hundred'],
      dtype='object')

In [59]:
# train 80% and test 20%
train = sample[:int(0.8*(len(sample)))]
test = sample[int(0.8*(len(sample))):]

dates = sample['order_time']
data = sample[['item_quantity', 'item_price',
               'item_disc_bins_item_disc_null', 
               'item_disc_bins_item_disc_sub_twenty',
               'item_disc_bins_item_disc_sub_forty',
               'item_disc_bins_item_disc_sub_sixty',
               'item_disc_bins_item_disc_sub_eighty',
               'item_disc_bins_item_disc_sub_hundred']]

data = data.astype(float)
data.index = pd.DatetimeIndex(dates)

# import VAR model and fit 
from statsmodels.tsa.vector_ar.var_model import VAR

model = VAR(endog=train)
model_fit = model.fit()

# make prediction on validation
prediction = model_fit.forecast(model_fit.y, steps=len(test))

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).